In [1]:
import numpy as np
import pandas as pd
from Utils import *
import re
from tqdm import tqdm
#crawler
import requests
from bs4 import BeautifulSoup

In [2]:
# 讀入官方的data
df = pd.read_csv('./data/tbrain_train_final_0610.csv')  

In [3]:
df[0:10]

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]


In [7]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind, count = np.unique(webKind, return_counts = True)
dict = {"name": webKind,  
        "counts": count
       }
webKind_df = pd.DataFrame(dict)
webKind_df

,name,counts
0,ccc.technews.tw,10
1,domestic.judicial.gov.tw,30
2,ec.ltn.com.tw,6
3,ent.ltn.com.tw,2
4,estate.ltn.com.tw,1
5,finance.technews.tw,56
6,hk.on.cc,57
7,house.ettoday.net,1
8,m.ctee.com.tw,530
9,m.ltn.com.tw,12


In [8]:
# 單獨測試
index = 3235
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
#context = getContext(category, url) 
context  = 0
print("url :", url, "\ncategory :", category, "\ncontext :", context)

url : https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=otc&i=337&h3370=%AD%BF%B7%4C%AC%EC%A7%DE&h3371=6270&h3372=20190807&h3373=170905&h3374=%A4%BD%A7%69%A5%BB%A4%BD%A5%71%C1%7E%B9%53%A9%65%AD%FB%B7%7C%A9%65%AD%FB%A5%F4%B4%C1%A9%A1%BA%A1%A7%EF%BF%EF&h3375=3&pgname=t05st02 
category : mops.twse.com.tw 
context : 0


In [9]:
def getContext(url):
    print("==========================")
    print("getContext url: ", url)
    soup, status_code = getSoup(url)
    contextStr = ""
    if status_code != 200:
        print("status_code: ", status_code, "url: ", url)
        contextStr = str(status_code)
    else:
        data = soup.find_all('p')
        for context in data:
            #print(context.text)
            contextStr += context.text
    return contextStr

In [10]:
context = getContext(url)

getContext url:  https://mops.twse.com.tw/mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=otc&i=337&h3370=%AD%BF%B7%4C%AC%EC%A7%DE&h3371=6270&h3372=20190807&h3373=170905&h3374=%A4%BD%A7%69%A5%BB%A4%BD%A5%71%C1%7E%B9%53%A9%65%AD%FB%B7%7C%A9%65%AD%FB%A5%F4%B4%C1%A9%A1%BA%A1%A7%EF%BF%EF&h3375=3&pgname=t05st02


In [11]:
df_addContext = df.copy()

In [12]:
df_addContext['webKind'] = df_addContext['hyperlink'].apply(lambda x: getWebsiteCategory(x))

In [58]:
# 原本想用lambda 不過這樣爬蟲會爬太快
#df_addContext['contextCrawler'] = df_addContext['hyperlink'].apply(lambda x: getContext(x))
contextList = []
hyperlinkList = df_addContext['hyperlink']
for index, url in enumerate(tqdm(hyperlinkList)):
    context = getContext(url)
    contextList.append(context)
df_addContext.at[index, "content_c"] = ""
for index, text in enumerate(tqdm(contextList)):
    df_addContext.at[index, "content_c"] = text

com/realtimenews/20190415001443-260402?chdtv
 62%|██████▏   | 3130/5023 [47:00<14:58,  2.11it/s]==========================
getContext url:  http://news.ltn.com.tw/news/society/paper/1257487
 62%|██████▏   | 3131/5023 [47:01<19:05,  1.65it/s]==========================
getContext url:  https://www.mirrormedia.mg/story/20191021wine002/
 62%|██████▏   | 3132/5023 [47:01<15:40,  2.01it/s]==========================
getContext url:  https://udn.com/news/story/7240/4029562
 62%|██████▏   | 3133/5023 [47:03<23:49,  1.32it/s]==========================
getContext url:  https://news.mingpao.com/ins/%e6%b8%af%e8%81%9e/article/20191003/s00001/1570077834987/%e3%80%90%e5%8d%81%e4%b8%80-%e7%a4%ba%e5%a8%81%e3%80%91%e4%b8%ad%e6%a7%8d%e7%94%b7%e7%94%9f%e8%a2%ab%e6%8e%a7%e6%9a%b4%e5%8b%95%e8%a5%b2%e8%ad%a6%e4%bb%8a%e5%8d%88%e6%8f%90%e5%a0%82
 62%|██████▏   | 3134/5023 [47:04<27:17,  1.15it/s]==========================
getContext url:  https://www.chinatimes.com/realtimenews/20190821004180-260402
 62%|█████

ConnectionError: HTTPSConnectionPool(host='mops.twse.com.tw', port=443): Max retries exceeded with url: /mops/web/t05st02?step=1&off=1&firstin=1&TYPEK=otc&i=337&h3370=%AD%BF%B7L%AC%EC%A7%DE&h3371=6270&h3372=20190807&h3373=170905&h3374=%A4%BD%A7i%A5%BB%A4%BD%A5q%C1~%B9S%A9e%AD%FB%B7%7C%A9e%AD%FB%A5%F4%B4%C1%A9%A1%BA%A1%A7%EF%BF%EF&h3375=3&pgname=t05st02 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11e086278>: Failed to establish a new connection: [Errno 61] Connection refused'))


100%|██████████| 3235/3235 [00:00<00:00, 95877.43it/s]

In [13]:
df_addContext.head(50)


,news_ID,hyperlink,content,name,webKind
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[],news.cnyes.com
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[],udn.com
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[],www.mirrormedia.mg
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[],www.chinatimes.com
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],domestic.judicial.gov.tw
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[],www.coolloud.org.tw
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[],m.ctee.com.tw
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[],www.mirrormedia.mg
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[],www.chinatimes.com
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[],news.cnyes.com


In [14]:
df_addContext.to_csv('./data/Result.csv')